In [2]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from functools import partial
from zhipuai import ZhipuAI


In [15]:

client = ZhipuAI(api_key="535af024d07df4663facbf5af957b179.aAxEiUuegF8HcETN")

prompt_learning="""
您是擅长情感分析与文本数据挖掘的数据分析师，请对社交媒体帖子进行空间认同感分析。请你先学习我提供的工作规则。
工作流程如下：
1.空间认同包含三个维度：空间满足感、空间意象感和空间归属感。请始终依照该定义进行判断推理。
空间满足感，是个体对于空间的满足感。主要体现在对环境容量和设施数量质量的评价，关注充足性和便利性。这意味着个体对周围环境的舒适程度和便捷性产生的认知。
空间意象感是个体对一个空间的整体印象,它在认知过程中逐步丰富和完善。包含三个维度：可辨别性、独特性与代表性、可意象性。可辨别性：能让人意识到自己在这个地方的位置和方向。让人对地方的某些组成部分和要素感到初步了解并且能够描述它们；独特性与代表性：能让人阐述地方的独特性与代表性；可意象性：对地方的感知和认知达到一定程度，能让人在脑海中重建对地方的相对完整意象。
空间归属感度量个体与空间的情感强度。包含空间的推荐性、依恋性、支持发展性、承诺与奉献性。推荐性：地方是可供推荐前往的。依恋性：对地方感到亲切，地方让人产生深厚的情感依恋与美好回忆，对地方忠诚与虔诚。支持发展性：个体愿意通过行为参与地方活动，从而保护地方环境、促进地方发展。承诺与奉献性：对地方产生深刻的承诺，愿意为了地方牺牲和奉献。
2.以下是空间认同的三个维度评估过程。
首先,请判断每条帖子是否包含上述定义的空间认同内容。如果没有具体描述或明确表达,请标记为0。
如果帖子包含空间认同内容,请分别给出对于空间满足感、空间意象感和空间归属感三个维度的评分。评分范围为-3到3的整数。其中,-3代表“极不认同”,-2代表“一般不认同”,-1代表“略微不认同”,1代表“略微认同”,2代表“一般认同”,3代表“极为认同”。
3.请您参照我给的例子“對於第一次上深圳玩既人，福田區商場最適合，因為全部有地下通道相連，而且由羅湖口岸以及福田口岸出發，都相當方便,强烈推荐你们下次来”，理解我的打分理念与输出格式（输出为markdown语言编写的表格）。
| 帖子序号|空间满足感评分| 空间满足感评分的推理过程|空间意象感评分|空间意象评分的推理过程 |空间归属感评分|空间归属感评分的推理过程|
|------|-------|--------|---------|---------|--------------|-------------|-----------|
| 1 | 3 |帖子提到福田區商场的便利性和环境容量，这表明了对该空间的满足感。 | 2 | 帖子通过描述地下通道的连通性，提供了对福田区商场的一定意象感。| 3 | 帖子强烈推荐福田区商场给初次访问深圳的人，显示了高程度的空间推荐性.| 

4.注意事项：（1）请严格遵守我对于空间认同三个维度的定义。（2）在回答后，请检查并修正任何可能的错误。如果需要，可以重新思考并进行必要的修改.（3）确保你的回答在评价空间认同的三个维度评估过程时是自洽的，避免矛盾。 
"""

# 异步运行同步函数的辅助函数
async def run_in_executor(executor, func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

# 进行API调用的同步函数
def call_api_sync(client, post_prompt, post_id):
    try:
        glm_completion = client.chat.completions.create(
            model="glm-3-turbo",
            messages=[{"role": "user", "content": post_prompt}]
        )
        response = glm_completion.choices[0].message.content if glm_completion.choices else "No response"
        return (post_id, post_prompt, response)
    except Exception as e:
        print(f"An error occurred for post {post_id}: {e}")
        return (post_id, post_prompt, "Error")

# 主异步函数，处理所有博文
async def process_posts(sample):
    results_df = pd.DataFrame(columns=["博文", "分析结果", "id"])
    na_df = []
    ad_df = []

    with ThreadPoolExecutor(max_workers=39) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for index, post in sample.iterrows():
            post_id = post['id']  
            post_prompt = f"{prompt_learning}请根据以上评分标准，对以下博文进行空间认同感分析：\n{post['全文内容']}\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅使用markdown表格形式输出打分内容即可"
            task = loop.create_task(run_in_executor(executor, call_api_sync, client, post_prompt, post_id))
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)
        for post_id, post_prompt, response in responses:
            if response not in ["No response", "Error"]:
                results_df = pd.concat([results_df, pd.DataFrame({'博文': [post_prompt], '分析结果': [response], 'id': [post_id]})], ignore_index=True)
                if response == "1":
                    ad_df.append(post_id)
            else:
                na_df.append(post_id)

    results_df.to_csv('glm-3_space_analysis_6.csv')
    print(results_df.head()) 

In [ ]:
test_post=pd.read_csv('D:/nly/python_code/glm-3_ad_clean_10w_50w.csv')
print(len(test_post))


In [16]:
sample = test_post[(test_post['id'] >= 300001) & (test_post['id'] <= 400000)]

In [ ]:

loop = asyncio.get_event_loop()

if not loop.is_running():
    loop.run_until_complete(process_posts(sample))
else:
    task = loop.create_task(process_posts(sample))
